# 🚀 VEC系统 - RSU计算资源对比实验（Colab）

## 📋 实验配置
- **实验类型**: RSU计算资源敏感性分析
- **训练轮次**: 1500 episodes（确保TD3充分收敛）
- **随机种子**: 42
- **预计时长**: 约2-3小时（取决于GPU类型）

## ⚙️ 使用前准备
1. **菜单栏**: 代码执行程序 → 更改运行时类型 → 选择 **T4 GPU** 或 **A100**
2. 依次运行下面的代码单元

## 📦 步骤1：挂载Google Drive（可选）
如果需要保存结果到云盘，运行此单元

In [ ]:
from google.colab import drive
import os

# 挂载Google Drive
drive.mount('/content/drive')

# 创建结果保存目录
save_dir = '/content/drive/MyDrive/VEC_results'
os.makedirs(save_dir, exist_ok=True)
print(f'✅ Google Drive已挂载，结果将保存到: {save_dir}')

## 🔧 步骤2：上传项目代码
**两种方式选一种**：

### 方式A：从GitHub克隆（推荐）

In [ ]:
# 如果你的代码已上传到GitHub，取消注释并修改仓库地址
# !git clone https://github.com/YOUR_USERNAME/VEC_mig_caching.git /content/VEC_mig_caching
# %cd /content/VEC_mig_caching
# print('✅ 项目代码克隆完成')

### 方式B：手动上传ZIP文件

In [ ]:
from google.colab import files
import zipfile
import shutil

# 1. 运行此单元，会弹出文件选择框
print('📤 请选择项目ZIP文件上传...')
uploaded = files.upload()

# 2. 解压项目
zip_name = list(uploaded.keys())[0]
with zipfile.ZipFile(zip_name, 'r') as zip_ref:
    zip_ref.extractall('/content')

# 3. 进入项目目录（根据实际压缩包结构调整）
project_dir = '/content/VEC_mig_caching'
if not os.path.exists(project_dir):
    # 尝试查找解压后的目录
    for item in os.listdir('/content'):
        if 'VEC' in item and os.path.isdir(f'/content/{item}'):
            shutil.move(f'/content/{item}', project_dir)
            break

%cd {project_dir}
print(f'✅ 项目已解压到: {os.getcwd()}')

## 🔨 步骤3：安装依赖

In [ ]:
# 安装必要的依赖包
!pip install flask-socketio -q
!pip install pyyaml -q

# 创建必要的目录结构
!mkdir -p results/td3_strategy_suite
!mkdir -p academic_figures
!mkdir -p logs

print('✅ 依赖安装完成')

## 🔍 步骤4：验证环境

In [ ]:
import torch
import sys

print('=' * 50)
print('🔍 环境信息')
print('=' * 50)
print(f'Python版本: {sys.version.split()[0]}')
print(f'PyTorch版本: {torch.__version__}')
print(f'CUDA可用: {torch.cuda.is_available()}')

if torch.cuda.is_available():
    print(f'GPU型号: {torch.cuda.get_device_name(0)}')
    print(f'GPU内存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB')
else:
    print('⚠️ 警告：未检测到GPU！请在菜单栏选择GPU运行时')

print('\n📂 当前工作目录:', os.getcwd())
print('\n📁 关键文件检查:')
critical_files = [
    'experiments/td3_strategy_suite/run_bandwidth_cost_comparison.py',
    'requirements.txt',
    'config/system_config.py'
]
for f in critical_files:
    status = '✅' if os.path.exists(f) else '❌'
    print(f'{status} {f}')

print('=' * 50)

## 🎯 步骤5：运行RSU计算资源对比实验

**实验说明**：
- 测试不同RSU计算资源配置下的系统性能
- 默认5个计算等级（从低到高）
- 每个配置训练1500轮（约30-40分钟/配置）

In [ ]:
# 🚀 开始实验
!python experiments/td3_strategy_suite/run_bandwidth_cost_comparison.py \
    --experiment-types rsu_compute \
    --rsu-compute-levels default \
    --episodes 1500 \
    --seed 42

### 💡 快速验证模式（可选）
如果只想快速验证代码能否运行，可以使用快速模式（500轮，约10分钟）

In [ ]:
# 快速验证模式（仅用于测试，不用于正式实验）
# !python experiments/td3_strategy_suite/run_bandwidth_cost_comparison.py \
#     --experiment-types rsu_compute \
#     --rsu-compute-levels default \
#     --episodes 500 \
#     --seed 42 \
#     --fast-mode

## 📊 步骤6：查看实验结果

In [ ]:
# 查看生成的结果文件
print('📁 实验结果文件：')
!ls -lh results/td3_strategy_suite/ | grep -E '(rsu_compute|summary)'

print('\n📊 生成的图表：')
!ls -lh academic_figures/ | tail -10

## 🖼️ 步骤7：显示关键图表

In [ ]:
from IPython.display import Image, display
import glob

# 显示最新生成的RSU计算资源对比图
rsu_figures = sorted(glob.glob('academic_figures/*rsu_compute*.png'))

if rsu_figures:
    print(f'📊 共找到 {len(rsu_figures)} 张图表\n')
    for fig_path in rsu_figures[-3:]:  # 显示最新的3张图
        print(f'\n{"=" * 60}')
        print(f'📈 {os.path.basename(fig_path)}')
        print(f'{"=" * 60}')
        display(Image(filename=fig_path))
else:
    print('⚠️ 未找到图表文件，请检查实验是否成功运行')

## 💾 步骤8：保存和下载结果

In [ ]:
from datetime import datetime
import shutil

# 创建带时间戳的结果包
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
result_archive = f'rsu_compute_results_{timestamp}'

# 打包结果
shutil.make_archive(
    result_archive, 
    'zip', 
    '.', 
    base_dir='results/td3_strategy_suite'
)

# 打包图表
figure_archive = f'rsu_compute_figures_{timestamp}'
shutil.make_archive(
    figure_archive, 
    'zip', 
    '.', 
    base_dir='academic_figures'
)

print(f'✅ 结果已打包：')
print(f'   - {result_archive}.zip')
print(f'   - {figure_archive}.zip')

# 如果挂载了Google Drive，复制到云盘
if os.path.exists('/content/drive/MyDrive'):
    save_dir = '/content/drive/MyDrive/VEC_results'
    shutil.copy(f'{result_archive}.zip', save_dir)
    shutil.copy(f'{figure_archive}.zip', save_dir)
    print(f'\n📤 已保存到Google Drive: {save_dir}')
else:
    print('\n💡 提示：运行下面的单元下载到本地')

In [ ]:
# 下载结果到本地
from google.colab import files

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
result_archive = f'rsu_compute_results_{timestamp}.zip'
figure_archive = f'rsu_compute_figures_{timestamp}.zip'

# 检查文件是否存在
for archive in [result_archive, figure_archive]:
    if os.path.exists(archive):
        print(f'📥 下载: {archive}')
        files.download(archive)
    else:
        print(f'⚠️ 文件不存在: {archive}')

## 📈 步骤9：查看关键指标摘要

In [ ]:
import json
import pandas as pd

# 查找最新的摘要文件
summary_files = sorted(glob.glob('results/td3_strategy_suite/*summary*.json'))

if summary_files:
    latest_summary = summary_files[-1]
    print(f'📊 读取摘要文件: {os.path.basename(latest_summary)}\n')
    
    with open(latest_summary, 'r', encoding='utf-8') as f:
        summary_data = json.load(f)
    
    # 提取关键指标
    if 'rsu_compute' in summary_data:
        print('🎯 RSU计算资源对比实验结果\n')
        print('=' * 80)
        
        for level_key, strategies in summary_data['rsu_compute'].items():
            print(f'\n📌 {level_key}')
            print('-' * 80)
            
            # 转换为DataFrame便于查看
            data = []
            for strategy, metrics in strategies.items():
                data.append({
                    '策略': strategy,
                    '平均时延(s)': f"{metrics.get('avg_delay', 0):.4f}",
                    '能耗(J)': f"{metrics.get('avg_energy_consumption', 0):.2f}",
                    '吞吐量(Mbps)': f"{metrics.get('avg_throughput_mbps', 0):.2f}",
                    'RSU利用率': f"{metrics.get('avg_rsu_utilization', 0):.2%}",
                    '卸载率': f"{metrics.get('avg_offload_ratio', 0):.2%}"
                })
            
            df = pd.DataFrame(data)
            print(df.to_string(index=False))
        
        print('\n' + '=' * 80)
    else:
        print('⚠️ 摘要文件中未找到RSU计算资源实验数据')
else:
    print('⚠️ 未找到摘要文件，请检查实验是否成功完成')

## 💡 使用技巧

1. **保持连接**：Colab会话超时时间约12小时，长时间实验建议定期点击页面防止断线
2. **GPU类型**：T4 GPU约3-4小时，A100约1.5-2小时
3. **中断恢复**：如果中断，已保存的结果在`results/`目录下
4. **监控进度**：实验运行时会实时输出进度条和关键指标
5. **内存管理**：如果遇到内存不足，可以重启运行时并从步骤3开始

## 📞 问题排查

- **找不到文件**: 确保项目目录结构完整，检查步骤2是否成功
- **GPU不可用**: 菜单栏 → 代码执行程序 → 更改运行时类型 → 选择GPU
- **训练太慢**: 检查是否使用了GPU，确认GPU型号
- **内存溢出**: 尝试减少episodes数量或使用A100 GPU

---
**祝实验顺利！** 🎉